# Using `faiss` to speed up similarity computation

* Area: Data-driven prototypes.
* Purpose: Use `faiss` to demonstrate professional-level retrieval performance, as an alternative to vector databases.




## Part I: Faiss demo
Load hotel review data. 

In [1]:
import pandas as pd

df = pd.read_csv('~/data/Hotel_Reviews.csv')
df.head(1)

,Hotel_Address,Additional_Number_of_Scoring,Review_Date,Average_Score,Hotel_Name,Reviewer_Nationality,Negative_Review,Review_Total_Negative_Word_Counts,Total_Number_of_Reviews,Positive_Review,Review_Total_Positive_Word_Counts,Total_Number_of_Reviews_Reviewer_Has_Given,Reviewer_Score,Tags,days_since_review,lat,lng
0,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,8/3/2017,7.7,Hotel Arena,Russia,I am so angry that i made this post available...,397,1403,Only the park outside of the hotel was beauti...,11,7,2.9,"[' Leisure trip ', ' Couple ', ' Duplex Double...",0 days,52.360576,4.915968


Obtain document-term matrix.

In [2]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(min_df=10, max_df = 0.8, stop_words='english')
vectors = vectorizer.fit_transform(df.Positive_Review)
vectors.shape

(515738, 8783)

For this demo, we use an older embedding method called matrix factorization, as it gives smaller embeddings (of size 20) so that we can run this example in class.

In [3]:
# Perform matrix factorization
from sklearn.decomposition import NMF
import sklearn.preprocessing as pre

nmf = NMF(n_components=20)
U = nmf.fit_transform(vectors)
# V = nmf.components_

# We normalize U to have unit L2 norm, so that cosine 
# similarity is the same as dot product
U = pre.normalize(U, norm='l2')

Use faiss for retrieval.

In [4]:
import faiss
index = faiss.IndexFlatIP(20)   # build the index
index.add(U)                  # add vectors to the index
index.ntotal

515738

In [5]:
distances, indices = index.search(U[:5], 2)
distances, indices

(array([[1.        , 0.99968314],
        [1.        , 0.94674593],
        [1.        , 0.9998525 ],
        [1.        , 0.95074403],
        [1.0000001 , 0.9999742 ]], dtype=float32),
 array([[     0, 181874],
        [     1,  43327],
        [     2, 486280],
        [     3, 371144],
        [     4, 175587]]))

## Part II: Performance Evaluation

Let us benchmark the time needed to perform the similarity operations under different settings: 

a) Using faiss as shown above

In [6]:
%%timeit
distances, indices = index.search(U[:5], 2)

4.46 ms ± 335 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


b) Doing the same operation in numpy:

In [7]:
import numpy as np

In [8]:
%%timeit
distances = np.dot(U[:5], U.T)
indices = np.argsort(distances, axis=1)[:,:2]

314 ms ± 23.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


c) Using faiss with an inverted index and a quantizer (the first speeds up the search, the second compresses the data).

In [9]:
quantizer = faiss.IndexFlatL2(20)  # this remains the same

d = 20        # Dimension (length) of vectors.
nlist = 100  # Number of inverted lists (number of partitions or cells).
nsegment = 5  # Number of segments for product quantization (number of subquantizers).
nbit = 4       # Number of bits to encode each segment.

index_pq = faiss.IndexIVFPQ(quantizer, d, nlist, nsegment, nbit)

index_pq.train(U)
index_pq.add(U)

In [10]:
%%timeit
distances, indices = index_pq.search(U[:5], 2)

93.9 µs ± 1.35 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


The conclusion is that faiss is faster than numpy dot. And that adding tricks to faiss, such as an inverted index and compression speeds things up even further. 

The latter comes at the expense of longer time for indexing, and non-exact search. 